3The first thing to be done is to gather the data from out datasets.

In [2]:
import pandas as pd

# Load dataset files
print("Loading training datasets...")

# Depression (train)
train_data_depression = pd.DataFrame(pd.read_json("../datasets/training/training_depression_40000.jsonl", lines=True))
train_data_depression["depression_related"] = [1] * len(train_data_depression.index)  # Dep. identifier: true
dep_size = len(train_data_depression.index)

# Non-depression (train)
train_data_control = pd.DataFrame(pd.read_json("../datasets/training/training_control_40000.jsonl", lines=True))
train_data_control["depression_related"] = [0] * len(train_data_control.index)  # Dep. identifier: false
non_dep_size = len(train_data_control.index)

# Join both data-frames
train = train_data_depression.append(train_data_control, ignore_index=True)
print("Datasets succesfully loaded and merged")

Loading training datasets...
Datasets succesfully loaded and merged


After that we have to work with our dataframe to have the correct data to work with. We will need:
* The text in the title and in the selftext of each post
* The proportion of pronouns in each post
* The hour of posting
* The month of posting

In [3]:
# Initialize the texts
try:
    train["selftext"]
except KeyError:
    train["text"] = train["title"]
else:
    train["text"] = train["title"] + train["selftext"]

In [7]:
import mlearning_utils as mlu
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1,2), stop_words="english", lowercase=True, preprocessor=mlu.pre_process)
cv.fit_transform(train["text"])

D:\Python38\lib\site-packages\sklearn\feature_extraction\text.py:382: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  warnings.warn('You

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [9]:
from nltk.tokenize import word_tokenize
# Pronouns proportion
train["pp1"] = train["text"].apply(lambda x: mlu.get_pronoun_proportion(word_tokenize(x), "pp1"))
train["pp2"] = train["text"].apply(lambda x: mlu.get_pronoun_proportion(word_tokenize(x), "pp2"))
train["pp3"] = train["text"].apply(lambda x: mlu.get_pronoun_proportion(word_tokenize(x), "pp3"))

TypeError: expected string or bytes-like object

In [6]:
import date_utils
# Hour and month
train["hour"] = train["created_utc"].apply(lambda x: date_utils.extract_field_from_timestamp(x, "hour"))
train["month"] = train["created_utc"].apply(lambda x: date_utils.extract_field_from_timestamp(x, "month"))